Aula 3 - PRÁTICA - Thais Damasceno - 11721BCC007

1) Carregue o dataset Iris.

In [31]:
from sklearn.datasets import load_iris
from pandas import DataFrame
import pandas as pd

# Carregar o dataset
iris = load_iris()

# DataFrame para melhor visualização
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

print(iris_df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


2) Aplique o classificador SVM.

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Separa as features (X) e o target (y)
X = iris_df.drop('target', axis=1)
y = iris_df['target']

# Divide os dados em treino e teste (70% treino 30% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inicializa e treina o classificador SVM com kernel linear
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

# Faz previsões no conjunto de teste
y_pred_linear = svm_linear.predict(X_test)

# Calcula a acurácia
accuracy_linear = accuracy_score(y_test, y_pred_linear)
print(f"Acurácia do SVM com kernel linear: {accuracy_linear}")

Acurácia do SVM com kernel linear: 1.0


3) Faça uma tabela com os resultados de acurácia do SVM para diferentes kernels. Para os kernels polinomiais e RBF, varie os parâmetros “degree” e “gamma”, respectivamente.
- Consulte a documentação: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html


In [33]:
# Define os kernels e parâmetros a serem testados
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
degrees = [2, 3, 4]  # Graus do polinômio (kernel poly)
gammas = [0.1, 1, 10] # Valores do parâmetro gamma (kernel rbf)

results = []

# Treinar e testar os modelos
for kernel in kernels:
     # Se o kernel for poly (polinomial), testa diferentes graus do polinômio
    if kernel == 'poly':
        for degree in degrees:
            svm = SVC(kernel=kernel, degree=degree)  # Cria e treina o modelo SVM com o kernel polinomial e o grau específico
            svm.fit(X_train, y_train) # treina o modelo
            y_pred = svm.predict(X_test) # faz a previsão nos dados de teste
            accuracy = accuracy_score(y_test, y_pred)
            results.append({'kernel': kernel, 'degree': degree, 'gamma': 'N/A', 'accuracy': accuracy})
    # Se o kernel for rbf (Radial Basis Function), testa diferentes valores de gamma
    elif kernel == 'rbf':
        for gamma in gammas:
            svm = SVC(kernel=kernel, gamma=gamma)
            svm.fit(X_train, y_train)
            y_pred = svm.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            results.append({'kernel': kernel, 'degree': 'N/A', 'gamma': gamma, 'accuracy': accuracy})
    # Para os kernels linear e sigmoid, não há necessidade de testar degree ou gamma
    else:
        svm = SVC(kernel=kernel)
        svm.fit(X_train, y_train)
        y_pred = svm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results.append({'kernel': kernel, 'degree': 'N/A', 'gamma': 'N/A', 'accuracy': accuracy})

# Resultados
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

 kernel degree gamma  accuracy
 linear    N/A   N/A  1.000000
   poly      2   N/A  1.000000
   poly      3   N/A  0.977778
   poly      4   N/A  1.000000
    rbf    N/A   0.1  1.000000
    rbf    N/A     1  1.000000
    rbf    N/A    10  0.977778
sigmoid    N/A   N/A  0.222222


4) Carregue o dataset Brasileirão Serie A
https://www.kaggle.com/datasets/adaoduque/campeonato-brasileiro-de-futebol?resource=download


In [34]:
import pandas as pd
from google.colab import drive

# Conectar com Google Drive
drive.mount('/content/drive', force_remount=True)


# Carregar os dados
df = pd.read_csv('/content/drive/MyDrive/campeonato-brasileiro-full.csv')

print(df.head())

Mounted at /content/drive
   ID  rodata        data   hora       mandante    visitante  \
0   1       1  29/03/2003  16:00        Guarani        Vasco   
1   2       1  29/03/2003  16:00   Athletico-PR       Gremio   
2   3       1  30/03/2003  16:00       Flamengo     Coritiba   
3   4       1  30/03/2003  16:00          Goias     Paysandu   
4   5       1  30/03/2003  16:00  Internacional  Ponte Preta   

  formacao_mandante formacao_visitante tecnico_mandante tecnico_visitante  \
0               NaN                NaN              NaN               NaN   
1               NaN                NaN              NaN               NaN   
2               NaN                NaN              NaN               NaN   
3               NaN                NaN              NaN               NaN   
4               NaN                NaN              NaN               NaN   

       vencedor              arena  mandante_Placar  visitante_Placar  \
0       Guarani     Brinco de Ouro                4  

5) Aplique o classificador Naive Bayes. Escolha algumas colunas (pelo menos Mandante e Visitante são essenciais) como entradas para predizer o resultado da partida.


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Remover espaços extras dos nomes das colunas
df.columns = df.columns.str.strip()

# Verificar os nomes das colunas
print("Colunas disponíveis:", df.columns)

# Selecionar colunas relevantes (mandante, visitante e vencedor)
cols = ["mandante", "visitante", "vencedor"]

print("\n")

df = df[cols].dropna()

# Verificar distribuição das classes
print("Distribuição das classes:")
print(df["vencedor"].value_counts().to_string())

# Transformar os times em valores numéricos
encoder = LabelEncoder()
df["mandante"] = encoder.fit_transform(df["mandante"])
df["visitante"] = encoder.fit_transform(df["visitante"])
df["vencedor"] = encoder.fit_transform(df["vencedor"])

# Definir variáveis de entrada e saída
X = df[["mandante", "visitante"]]
y = df["vencedor"]

# Dividir em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar o classificador Naive Bayes
nb = GaussianNB()

# Mostrar o número de vitórias de cada time
_ = nb.fit(X_train, y_train)

Colunas disponíveis: Index(['ID', 'rodata', 'data', 'hora', 'mandante', 'visitante',
       'formacao_mandante', 'formacao_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'vencedor', 'arena', 'mandante_Placar',
       'visitante_Placar', 'mandante_Estado', 'visitante_Estado'],
      dtype='object')


Distribuição das classes:
vencedor
-                  2221
Sao Paulo           365
Flamengo            354
Santos              340
Internacional       339
Palmeiras           329
Atletico-MG         327
Corinthians         326
Fluminense          322
Gremio              316
Athletico-PR        309
Cruzeiro            304
Botafogo-RJ         247
Goias               209
Vasco               208
Coritiba            189
Figueirense         142
Sport               131
Bahia               127
Vitoria             123
Ponte Preta         114
Fortaleza           108
Juventude            85
Parana               83
Atletico-GO          72
Ceara                72
Chapecoense          70
Ava

6) Use os métodos predict(X) e predict_proba(X) para testar os resultados
- Consulte a documentação:		https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [36]:
from sklearn.metrics import classification_report

# Previsões usando predict e predict_proba
y_pred = nb.predict(X_test)
y_prob = nb.predict_proba(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, zero_division=1))


Acurácia: 0.27483640690065436

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.27      1.00      0.43       456
           1       1.00      0.00      0.00        10
           2       1.00      0.00      0.00         1
           3       1.00      0.00      0.00        68
           4       1.00      0.00      0.00         7
           5       1.00      0.00      0.00        70
           6       1.00      0.00      0.00        14
           7       1.00      0.00      0.00        24
           8       1.00      0.00      0.00         1
           9       1.00      0.00      0.00        42
          10       1.00      0.00      0.00         9
          11       1.00      0.00      0.00         3
          12       1.00      0.00      0.00         2
          13       1.00      0.00      0.00        14
          14       1.00      0.00      0.00        16
          15       1.00      0.00      0.00        55
          16       1.0

7) Repita o passo 5, mas agora tente apenas prever entre vitória ou não-vitória (isto é, considerando empate ou derrota como a mesma classe)


In [37]:
import numpy as np

# Prevê apenas vitória ou não-vitória
# 1 representa que o time mandante venceu a partida
# 0 representa derrota ou empate
df["Vitoria"] = np.where(df["vencedor"] == df["mandante"], 1, 0)
y_binary = df["Vitoria"]

# Verifica distribuição das classes no modelo binário
print("Distribuição das classes (Vitória vs. Não-Vitória):\n")
print(df["Vitoria"].value_counts().to_string())

# Divide novamente em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Treina novamente o Naive Bayes
nb.fit(X_train, y_train)
y_pred_binary = nb.predict(X_test)

# Avaliação do modelo binário
print("\nAcurácia (Vitória vs. Não-Vitória):", accuracy_score(y_test, y_pred_binary))
print("\nRelatório de Classificação (Vitória vs. Não-Vitória):\n")
print(classification_report(y_test, y_pred_binary, zero_division=1))

Distribuição das classes (Vitória vs. Não-Vitória):

Vitoria
0    8323
1      82

Acurácia (Vitória vs. Não-Vitória): 0.992266508030934

Relatório de Classificação (Vitória vs. Não-Vitória):

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1668
           1       1.00      0.00      0.00        13

    accuracy                           0.99      1681
   macro avg       1.00      0.50      0.50      1681
weighted avg       0.99      0.99      0.99      1681

